# Setup

In [40]:
from langchain_groq import ChatGroq
from langchain.cache import InMemoryCache,SQLiteCache
from langchain.globals import set_llm_cache
import os
import json
import hashlib


In [41]:
from dotenv import find_dotenv, load_dotenv

load_dotenv(find_dotenv())

True

In [42]:
groq = ChatGroq(model_name='llama3-8b-8192')
set_llm_cache(InMemoryCache())

# Memória

In [43]:
prompt = 'Me diga em poucas palavras que foi Carl Sagan.'
response1 = groq.invoke(prompt)
print("Primeira resposta (API chamada):", response1)

Primeira resposta (API chamada): content='Carl Sagan foi um astrofísico e cosmológico estadunidense, autor e apresentador de TV. Ele foi um defensor da ciência e da razão, e sua obra "Cosmos" é uma das mais influentes sobre a astronomia e a filosofia da ciência.' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 67, 'prompt_tokens': 24, 'total_tokens': 91, 'completion_time': 0.08358044, 'prompt_time': 0.004073822, 'queue_time': 0.145892772, 'total_time': 0.087654262}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_343314801a', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None} id='run--9ca5102b-3cfd-4008-aa03-633a3ef3de92-0' usage_metadata={'input_tokens': 24, 'output_tokens': 67, 'total_tokens': 91}


In [44]:
response2 = groq.invoke(prompt)
print("Segunda resposta (usando cache):", response2)

Segunda resposta (usando cache): content='Carl Sagan foi um astrofísico e cosmológico estadunidense, autor e apresentador de TV. Ele foi um defensor da ciência e da razão, e sua obra "Cosmos" é uma das mais influentes sobre a astronomia e a filosofia da ciência.' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 67, 'prompt_tokens': 24, 'total_tokens': 91, 'completion_time': 0.08358044, 'prompt_time': 0.004073822, 'queue_time': 0.145892772, 'total_time': 0.087654262}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_343314801a', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None} id='run--9ca5102b-3cfd-4008-aa03-633a3ef3de92-0' usage_metadata={'input_tokens': 24, 'output_tokens': 67, 'total_tokens': 91, 'total_cost': 0}


# Disco / Banco de Dados

In [45]:
set_llm_cache(SQLiteCache(database_path="groq_cache.db"))

In [46]:
prompt = 'Me diga em poucas palavras quem foi Neil Armstrong.'

In [47]:
response1 = groq.invoke(prompt)
print("Primeira resposta (API chamada):", response1)

Primeira resposta (API chamada): content='Neil Armstrong foi um astronauta norte-americano que foi o primeiro homem a caminhar na Lua em 20 de julho de 1969, durante a missão Apollo 11.' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 42, 'prompt_tokens': 23, 'total_tokens': 65, 'completion_time': 0.027953195, 'prompt_time': 0.007863625, 'queue_time': 0.145866529, 'total_time': 0.03581682}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_0fb809dba3', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None} id='run--f447f9b3-45b2-4add-8b5e-f4d0d7b69828-0' usage_metadata={'input_tokens': 23, 'output_tokens': 42, 'total_tokens': 65, 'total_cost': 0}


In [48]:
response2 = groq.invoke(prompt)
print("Segunda resposta (usando cache):", response2)

Segunda resposta (usando cache): content='Neil Armstrong foi um astronauta norte-americano que foi o primeiro homem a caminhar na Lua em 20 de julho de 1969, durante a missão Apollo 11.' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 42, 'prompt_tokens': 23, 'total_tokens': 65, 'completion_time': 0.027953195, 'prompt_time': 0.007863625, 'queue_time': 0.145866529, 'total_time': 0.03581682}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_0fb809dba3', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None} id='run--f447f9b3-45b2-4add-8b5e-f4d0d7b69828-0' usage_metadata={'input_tokens': 23, 'output_tokens': 42, 'total_tokens': 65, 'total_cost': 0}


# Personalizado

In [49]:
class SimpleDiskCache:
    def __init__(self, cache_dir='cache_dir'):
        self.cache_dir = cache_dir
        os.makedirs(self.cache_dir, exist_ok=True)

    def _get_cache_path(self, key):
        hashed_key = hashlib.md5(key.encode()).hexdigest() #hasg cria nome de arquivo único
        return os.path.join(self.cache_dir, f"{hashed_key}.json")

    def lookup(self, key, llm_string):
        cache_path = self._get_cache_path(key)
        if os.path.exists(cache_path):
            with open(cache_path, 'r') as f:
                return json.load(f)
        return None

    def update(self, key, value, llm_string):
        cache_path = self._get_cache_path(key)
        with open(cache_path, 'w') as f:
            #json.dump(value.to_json(), f)
            print(value)

In [50]:
cache = SimpleDiskCache()
set_llm_cache(cache)
prompt = 'Me diga em poucas palavras quem foi Neil Degrasse Tyson.'

In [51]:
def invoke_with_cache(llm, prompt, cache):
    cached_response = cache.lookup(prompt, "")
    if cached_response:
        print("Usando cache:")
        return cached_response

    response = llm.invoke(prompt)
    cache.update(prompt, response, "")
    return response

In [52]:
response1 = invoke_with_cache(groq, prompt, cache)
response_text1 = response1.replace('\n', ' ') 

print("Primeira resposta (API chamada):", response_text1)

{"id": ["langchain_groq", "chat_models", "ChatGroq"], "kwargs": {"groq_api_key": {"id": ["GROQ_API_KEY"], "lc": 1, "type": "secret"}, "max_retries": 2, "model_name": "llama3-8b-8192", "n": 1, "service_tier": "on_demand", "temperature": 0.7}, "lc": 1, "name": "ChatGroq", "type": "constructor"}---[('stop', None)]
content='Neil deGrasse Tyson é um astrofísico, escritor e apresentador de TV americano. Ele é conhecido por sua capacidade de explicar complexos conceitos científicos de forma acessível ao público em geral.' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 50, 'prompt_tokens': 26, 'total_tokens': 76, 'completion_time': 0.062288931, 'prompt_time': 0.004261189, 'queue_time': 0.145108745, 'total_time': 0.06655012}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_343314801a', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None} id='run--b5f8b153-0021-478a-ba96-fc14ab15368a-0' usage_metadata={'input_tokens': 26, 'output_tokens'

AttributeError: 'AIMessage' object has no attribute 'replace'

In [ ]:
response2 = invoke_with_cache(groq, prompt, cache)
response_text2 = response2.replace('\n', ' ')  

print("Segunda resposta (usando cache):", response_text2)